In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import nltk
from sklearn import svm

In [14]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from IPython.display import display

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\moham\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\moham\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\moham\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\moham\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\omw-1.4.zip.


True

In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier

In [16]:
data = pd.read_csv("EronDataset.csv")

In [17]:
tokenized_messages = data['text'].str.lower().apply(word_tokenize)
print(tokenized_messages)

0       [subject, :, enron, methanol, ;, meter, #, :, ...
1       [subject, :, hpl, nom, for, january, 9, ,, 200...
2       [subject, :, neon, retreat, ho, ho, ho, ,, we,...
3       [subject, :, photoshop, ,, windows, ,, office,...
4       [subject, :, re, :, indian, springs, this, dea...
                              ...                        
5166    [subject, :, put, the, 10, on, the, ft, the, t...
5167    [subject, :, 3, /, 4, /, 2000, and, following,...
5168    [subject, :, calpine, daily, gas, nomination, ...
5169    [subject, :, industrial, worksheets, for, augu...
5170    [subject, :, important, online, banking, alert...
Name: text, Length: 5171, dtype: object


In [18]:
def alpha(tokens):
    alpha = []
    for token in tokens:
        if str.isalpha(token) or token in ['n\'t','won\'t']:
            if token == 'n\'t':
                alpha.append('not')
                continue
            elif token == 'won\'t':
                alpha.append('wont')
                continue
            alpha.append(token)
    return alpha

tokenized_messages = tokenized_messages.apply(alpha)
print(tokenized_messages)

0       [subject, enron, methanol, meter, this, is, a,...
1       [subject, hpl, nom, for, january, see, attache...
2       [subject, neon, retreat, ho, ho, ho, we, re, a...
3       [subject, photoshop, windows, office, cheap, m...
4       [subject, re, indian, springs, this, deal, is,...
                              ...                        
5166    [subject, put, the, on, the, ft, the, transpor...
5167    [subject, and, following, noms, hpl, can, t, t...
5168    [subject, calpine, daily, gas, nomination, jul...
5169    [subject, industrial, worksheets, for, august,...
5170    [subject, important, online, banking, alert, d...
Name: text, Length: 5171, dtype: object


In [19]:
def remove_stop_words(tokens):
    no_stop = []
    for token in tokens:
        if token not in stopwords.words('english'):
            no_stop.append(token)
    return no_stop
tokenized_messages = tokenized_messages.apply(remove_stop_words)
print(tokenized_messages)

0       [subject, enron, methanol, meter, follow, note...
1       [subject, hpl, nom, january, see, attached, fi...
2       [subject, neon, retreat, ho, ho, ho, around, w...
3       [subject, photoshop, windows, office, cheap, m...
4       [subject, indian, springs, deal, book, teco, p...
                              ...                        
5166    [subject, put, ft, transport, volumes, decreas...
5167    [subject, following, noms, hpl, take, extra, m...
5168    [subject, calpine, daily, gas, nomination, jul...
5169    [subject, industrial, worksheets, august, acti...
5170    [subject, important, online, banking, alert, d...
Name: text, Length: 5171, dtype: object


In [9]:
def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized = []
    for token in tokens:
        lemmatized.append(lemmatizer.lemmatize(token))
    return " ".join(lemmatized)

tokenized_messages = tokenized_messages.apply(lemmatize)
print(tokenized_messages)

0       subject enron methanol meter follow note gave ...
1       subject hpl nom january see attached file hpln...
2       subject neon retreat ho ho ho around wonderful...
3       subject photoshop window office cheap main tre...
4       subject indian spring deal book teco pvr reven...
                              ...                        
5166    subject put ft transport volume decreased cont...
5167    subject following noms hpl take extra mmcf wee...
5168    subject calpine daily gas nomination julie men...
5169    subject industrial worksheet august activity a...
5170    subject important online banking alert dear va...
Name: text, Length: 5171, dtype: object


In [12]:
x = data['text']
y = data['label_num']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2, random_state=34, stratify=y)

# Create the tf-idf vectorizer
vectorizer = TfidfVectorizer(strip_accents='ascii')

# First fit the vectorizer with our training set
tfidf_train = vectorizer.fit_transform(x_train)

# Now we can fit our test data with the same vectorizer
tfidf_test = vectorizer.transform(x_test)


  (0, 5682)	0.15404804950329298
  (0, 5119)	0.16684788059401068
  (0, 19572)	0.21212760464058913
  (0, 2765)	0.24580013172945345
  (0, 352)	0.22166431330287345
  (0, 16819)	0.08319260674607103
  (0, 1)	0.10219606034575256
  (0, 801)	0.2712724151456959
  (0, 22345)	0.17866567260923577
  (0, 39303)	0.1877205765921573
  (0, 41982)	0.1519475499877675
  (0, 44269)	0.060022938163181314
  (0, 13782)	0.13829305858675012
  (0, 5546)	0.11636556812018235
  (0, 26889)	0.14246337705560963
  (0, 18780)	0.1001168326455955
  (0, 29671)	0.16118303780810428
  (0, 16660)	0.15383251377127008
  (0, 40927)	0.2054275177069678
  (0, 5128)	0.26203692732535905
  (0, 9098)	0.16841189266960738
  (0, 39824)	0.07680294838626842
  (0, 35783)	0.1619790704555782
  (0, 36795)	0.1519475499877675
  (0, 5816)	0.11783699478368746
  :	:
  (4134, 43220)	0.2242773747784929
  (4134, 4722)	0.16865382007384766
  (4134, 7637)	0.1998046431770514
  (4134, 22667)	0.12855307181182754
  (4134, 21022)	0.06814013228586584
  (4134, 29534

In [24]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(tfidf_train, y_train)
pred_results = clf.predict(tfidf_test)
print("Confusion Matrix" + confusion_matrix(y_test,pred_results))
print(classification_report(y_test,pred_results))

[[718  17]
 [  7 293]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       735
           1       0.95      0.98      0.96       300

    accuracy                           0.98      1035
   macro avg       0.97      0.98      0.97      1035
weighted avg       0.98      0.98      0.98      1035

